<a href="https://colab.research.google.com/github/Ayanlola2002/Data-Science-Nigeria/blob/master/DSN_BOOTCAMP_2020_average_blendVerynew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [438]:
#mounting Gdrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [439]:
import pandas as pd
import numpy as np
#from sklearn.experimental import enable_iterative_imputer
#from sklearn.impute import IterativeImputer
#imp= IterativeImputer()
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV,StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMModel,LGBMClassifier
from sklearn.ensemble import ExtraTreesClassifier
!pip install catboost
from catboost import Pool, CatBoostClassifier, cv
!pip install vecstack
from vecstack import stacking
#pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [440]:
train_df=pd.read_csv('/content/drive/My Drive/DSN_BOOTCAMP/Train.csv')
test_df=pd.read_csv('/content/drive/My Drive/DSN_BOOTCAMP/Test.csv')

In [441]:

train_df.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,NaN,NaN,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,NaN,1052.0,8.0000,2.0,NaN,12.0,12.0,4.0,0.0,NaN,NaN,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.24,14584.29,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,NaN,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,NaN,NaN,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,1079864.0,NaN,NaN,0.0,16576457,22452691.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,1597.5,4746.0,3164.0,60.0,15696.0,NaN,14508.0,70.0,0.6666,0.0,NaN,12.0,0.0,6.0,0.0,149.6805,NaN,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,NaN,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.00,0.00,85.800,1701.0,4014.0,2676.0,60.0,11679.0,NaN,NaN,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,NaN,NaN,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.19,10326.45,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,NaN,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [442]:
test_df.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,0.0,1649749.0,5446.0,0.0,48187375,114686147.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,1086.0,6204.0,2676.0,60.0,14691.0,9794.0,NaN,238.0,1.8334,0.0,NaN,22.0,2.0,14.0,0.0,NaN,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,18308285.0,622789.0,1025793.0,1396510.0,34694910,87479487.0,1.0,1.0,1.0,2.0,2.0,142.560,NaN,NaN,570.585,3082.5,10221.0,6814.0,60.0,11955.0,3528.0,NaN,1216.0,19.5000,3.0,NaN,30.0,18.0,14.0,2.0,NaN,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,0.0,118256.0,NaN,NaN,21973443,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,852.0,4836.0,NaN,NaN,NaN,7.8334,0.0,NaN,2.0,NaN,0.0,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,493641.0,1391064.0,1176725.0,1220293.0,16576457,NaN,2.0,2.0,2.0,3.0,3.0,109.005,13050.00,NaN,109.005,1414.5,4197.0,2798.0,60.0,14964.0,NaN,NaN,248.0,9.0000,1.0,1.0,36.0,0.0,26.0,0.0,NaN,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,5752921.0,271133.0,613064.0,4779643.0,48187375,72049802.0,0.0,0.0,0.0,3.0,3.0,139.065,469.29,469.29,15.180,2724.0,9855.0,6570.0,182.0,11679.0,5900.0,NaN,2388.0,16.6666,1.0,0.0,24.0,28.0,20.0,0.0,NaN,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


In [443]:
Application_id=test_df['Applicant_ID']

In [444]:
 #train_df.dropna(thresh=train_df.shape[1]*0.7,how='all',axis=0,inplace=True)

In [445]:

print(train_df.shape)
print(test_df.shape)

(56000, 52)
(24000, 51)


In [446]:
train_df.shape

(56000, 52)

In [447]:
train_df.default_status.unique()

array(['no', 'yes'], dtype=object)

In [448]:
#target varriable
train_df.loc[train_df.default_status=='no','default_status']=0
#target varriable
train_df.loc[train_df.default_status=='yes','default_status']=1

In [449]:
#saving the order Id column and predictor column
#Application_id=test_df['Applicant_ID']
y_train=train_df.iloc[:,-1].astype('int')

In [450]:
y_train

0        0
1        0
2        1
3        0
4        0
        ..
55995    0
55996    1
55997    0
55998    0
55999    0
Name: default_status, Length: 56000, dtype: int64

In [451]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     53471 non-null  float64
 2   form_field2     52156 non-null  float64
 3   form_field3     55645 non-null  float64
 4   form_field4     55645 non-null  float64
 5   form_field5     55645 non-null  float64
 6   form_field6     42640 non-null  float64
 7   form_field7     50837 non-null  float64
 8   form_field8     42640 non-null  float64
 9   form_field9     47992 non-null  float64
 10  form_field10    55645 non-null  float64
 11  form_field11    24579 non-null  float64
 12  form_field12    46105 non-null  float64
 13  form_field13    50111 non-null  float64
 14  form_field14    56000 non-null  int64  
 15  form_field15    33525 non-null  float64
 16  form_field16    42964 non-null  float64
 17  form_field17    44849 non-null 

In [452]:
#filling field6 and field7 nan values with zeros
#train_df.fillna({'form_field6':0,'form_field7':0},inplace=True)
#test_df.fillna({'form_field6':0,'form_field7':0},inplace=True)
train_df.replace(np.nan,-1,inplace=True)
test_df.replace(np.nan,-1,inplace=True)

In [453]:
#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train_df.drop("Applicant_ID", axis = 1, inplace = True)
test_df.drop("Applicant_ID", axis = 1, inplace = True)

In [454]:
# let's delete the last column from the dataset to  concat train and test
train_df = train_df.drop(['default_status'], axis = 1)

In [455]:
#cat_dims= np.where(train_df.dtypes != float)[0]
cat_dims=np.array([46])
cat_dims

array([46])

In [456]:
train_df.shape


(56000, 50)

In [457]:
test_df.shape

(24000, 50)

In [458]:
train_df.isnull().sum()

form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field11    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field23    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field31    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field40    0
form_field41    0
form_field42    0
form_field43    0
form_field44    0
form_field45    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [459]:
test_df.isnull().sum()

form_field1     0
form_field2     0
form_field3     0
form_field4     0
form_field5     0
form_field6     0
form_field7     0
form_field8     0
form_field9     0
form_field10    0
form_field11    0
form_field12    0
form_field13    0
form_field14    0
form_field15    0
form_field16    0
form_field17    0
form_field18    0
form_field19    0
form_field20    0
form_field21    0
form_field22    0
form_field23    0
form_field24    0
form_field25    0
form_field26    0
form_field27    0
form_field28    0
form_field29    0
form_field30    0
form_field31    0
form_field32    0
form_field33    0
form_field34    0
form_field35    0
form_field36    0
form_field37    0
form_field38    0
form_field39    0
form_field40    0
form_field41    0
form_field42    0
form_field43    0
form_field44    0
form_field45    0
form_field46    0
form_field47    0
form_field48    0
form_field49    0
form_field50    0
dtype: int64

In [460]:
# lets concat the train and test sets for preprocessing and visualizations
all_data = pd.concat([train_df, test_df], axis = 0)

# let's check the shape
all_data.shape

(80000, 50)

In [461]:
data_joined=all_data.copy()

In [462]:
data_joined.shape

(80000, 50)

In [463]:

data_joined

,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,form_field10,form_field11,form_field12,form_field13,form_field14,form_field15,form_field16,form_field17,form_field18,form_field19,form_field20,form_field21,form_field22,form_field23,form_field24,form_field25,form_field26,form_field27,form_field28,form_field29,form_field30,form_field31,form_field32,form_field33,form_field34,form_field35,form_field36,form_field37,form_field38,form_field39,form_field40,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,3436.0,0.28505,1.6560,0.0000,0.0000,0.0,10689720.0,252072.0,4272776.0,11333126.0,4397256.0,2301324.0,7875694.0,96375139,112181765.0,1.0,1.0,1.0,3.0,3.0,134.655,-1.000,-1.000,134.655,4000.5,17064.0,11376.0,60.0,17064.0,5962.0,-1.0,1052.0,8.0000,2.0,-1.0,12.0,12.0,4.0,0.0,-1.0000,-1.0000,0.392854,2.02,0.711632,0.0,0.0,charge,-1.000000,1.129518,0.044335
1,3456.0,0.67400,0.2342,0.0000,0.0000,0.0,898979.0,497531.0,9073814.0,2533168.0,244292.0,3982971.0,2141445.0,28141427,45169902.0,1.0,1.0,1.0,2.0,2.0,109.230,11385.240,14584.290,86.685,4161.0,13413.0,8942.0,304.0,13413.0,5780.0,-1.0,2554.0,17.5000,0.0,0.0,16.0,24.0,8.0,0.0,-1.0000,-1.0000,0.314281,8.08,0.183584,-1.0,0.0,charge,349.805730,1.620483,0.322436
2,3276.0,0.53845,3.1510,0.0000,6.2820,-1.0,956940.0,-1.0,192944.0,1079864.0,-1.0,-1.0,0.0,16576457,22452691.0,-1.0,-1.0,-1.0,0.0,0.0,-1.000,-1.000,-1.000,-1.000,1597.5,4746.0,3164.0,60.0,15696.0,-1.0,14508.0,70.0,0.6666,0.0,-1.0,12.0,0.0,6.0,0.0,149.6805,-1.0000,0.162965,18.18,0.791136,0.0,0.0,charge,-1.000000,1.513370,0.011640
3,3372.0,0.17005,0.5050,0.0000,0.0000,192166.0,3044703.0,385499.0,3986472.0,3621979.0,-1.0,771776.0,1186839.0,48187375,136297820.0,0.0,0.0,0.0,1.0,1.0,120.120,8580.000,0.000,85.800,1701.0,4014.0,2676.0,60.0,11679.0,-1.0,-1.0,522.0,7.8334,1.0,1.0,4.0,8.0,4.0,1.0,-1.0000,-1.0000,0.488884,2.02,0.685168,-1.0,0.0,charge,89.940100,0.664452,0.082729
4,3370.0,0.77270,1.1010,0.0000,0.0000,1556.0,214728.0,214728.0,1284089.0,361770.0,396391.0,818456.0,511535.0,28141427,15292757.0,0.0,0.0,0.0,3.0,3.0,104.535,8360.190,10326.450,115.335,1551.0,3285.0,2190.0,60.0,19437.0,12958.0,5202.0,272.0,1.0000,0.0,0.0,24.0,2.0,24.0,0.0,150.0135,-1.0000,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,3258.0,0.37925,0.5698,5.1282,0.0000,-1.0,2280318.0,-1.0,-1.0,2280318.0,1743887.0,-1.0,0.0,21973443,126128971.0,-1.0,-1.0,-1.0,0.0,0.0,-1.000,-1.000,-1.000,-1.000,547.5,-1.0,-1.0,304.0,3741.0,2068.0,-1.0,24.0,0.3334,0.0,0.0,6.0,0.0,6.0,0.0,-1.0000,92.6910,1.100000,0.00,0.800000,-1.0,1.0,charge,-1.000000,0.000000,-1.000000
23996,3514.0,0.95260,0.0000,0.0000,0.0000,1494538.0,4773419.0,4773419.0,89718571.0,12999991.0,-1.0,48989882.0,97696961.0,96375139,-1.0,0.0,0.0,0.0,3.0,3.0,123.195,11663.055,-1.000,123.195,1485.0,6387.0,4258.0,60772.0,14691.0,3286.0,-1.0,586.0,14.1666,0.0,0.0,10.0,2.0,8.0,0.0,-1.0000,125.0220,0.000000,12.12,0.196400,0.0,0.0,charge,4200.072959,3.000000,0.930931
23997,3302.0,0.05755,4.2394,0.0000,25.4376,-1.0,49403.0,-1.0,-1.0,49403.0,215895.0,-1.0,0.0,21973443,-1.0,-1.0,-1.0,0.0,0.0,1.0,-1.000,13080.000,-1.000,-1.000,2304.0,-1.0,-1.0,60.0,10038.0,-1.0,5109.0,202.0,5.3334,0.0,-1.0,8.0,4.0,2.0,0.0,-1.0000,-1.0000,1.100000,1.01,0.800000,-1.0,0.0,charge,-1.000000,4.855060,-1.000000
23998,3334.0,0.75670,1.4428,0.0000,0.0000,0.0,2513329.0,37733.0,393668.0,2551062.0,121757.0,355935.0,673748.0,16576457,-1.0,1.0,2.0,2.0,2.0,2.0,129.435,13024.995,13024.995,138.450,706.5,2463.0,1642.0,0.0,6297.0,2312.0,-1.0,62.0,2.6666,0.0,0.0,12.0,0.0,4.0,0.0,-1.0000,31.1955,0.440000,2.02,0.519776,-1.0,0.0,charge,27.325129,0.818733,0.023749


In [464]:
#label encoding categorical varriables

le = LabelEncoder()
data_joined['form_field47'] = le.fit_transform(data_joined['form_field47'])
# removing unnecassary columns

#data_joined =data_joined.drop(['form_field47'], axis = 1)

In [465]:

# separating train and test datasets from data

x_train = data_joined.iloc[:56000,:]
x_test = data_joined.iloc[56000:,:]

# checking the shape of train and test
print("Shape of train :", x_train.shape)
print("Shape of test :", x_test.shape)

Shape of train : (56000, 50)
Shape of test : (24000, 50)


In [466]:
# train and valid sets from train
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size = 0.2, random_state = 0)

# checking the shapes
print(x_train.shape)
print(y_train.shape)
print(x_valid.shape)
print(y_valid.shape)

(44800, 50)
(44800,)
(11200, 50)
(11200,)


In [467]:

# standardization techniques

'''from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_valid = sc.transform(x_valid)
x_test = sc.transform(x_test)'''

'from sklearn.preprocessing import StandardScaler\n\nsc = StandardScaler()\nx_train = sc.fit_transform(x_train)\nx_valid = sc.transform(x_valid)\nx_test = sc.transform(x_test)'

In [468]:
import hyperopt
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval

In [469]:
 '''  #LGBoost Tuning
   def hyperopt_lgb_score(params):
    clf = LGBMClassifier(**params)
    current_score = cross_val_score(clf,x_train,y_train, cv=StratifiedKFold(n_splits=10)).mean()
    print(current_score, params)
    return current_score 
 
space_lgb = {
            'learning_rate': hp.quniform('learning_rate', 0, 0.05, 0.0001),
            'n_estimators': hp.choice('n_estimators', range(100, 1000,100)),
            'max_depth':  hp.choice('max_depth', np.arange(2, 12, dtype=int)),
            'num_leaves': hp.choice('num_leaves', 2*np.arange(2, 2**11, dtype=int)),
            'min_child_weight': hp.quniform('min_child_weight', 1, 9, 0.025),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.005),
            'objective': 'binary',
            'boosting_type': 'gbdt',
            }
 
best = fmin(fn=hyperopt_lgb_score, space=space_lgb, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)'''

"  #LGBoost Tuning\n  def hyperopt_lgb_score(params):\n   clf = LGBMClassifier(**params)\n   current_score = cross_val_score(clf,x_train,y_train, cv=StratifiedKFold(n_splits=10)).mean()\n   print(current_score, params)\n   return current_score \n\nspace_lgb = {\n           'learning_rate': hp.quniform('learning_rate', 0, 0.05, 0.0001),\n           'n_estimators': hp.choice('n_estimators', range(100, 1000,100)),\n           'max_depth':  hp.choice('max_depth', np.arange(2, 12, dtype=int)),\n           'num_leaves': hp.choice('num_leaves', 2*np.arange(2, 2**11, dtype=int)),\n           'min_child_weight': hp.quniform('min_child_weight', 1, 9, 0.025),\n           'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.005),\n           'objective': 'binary',\n           'boosting_type': 'gbdt',\n           }\n\nbest = fmin(fn=hyperopt_lgb_score, space=space_lgb, algo=tpe.suggest, max_evals=10)\nprint('best:')\nprint(best)"

In [470]:
'''params = space_eval(space_lgb, best)
params'''

'params = space_eval(space_lgb, best)\nparams'

In [471]:
'''def hyperopt_xgb_score(params):
    clf = XGBClassifier(**params)
    current_score = cross_val_score(clf, x_train,y_train,cv=StratifiedKFold()).mean()
    print(current_score, params)
    return current_score 
 
space_xgb = {
            'learning_rate': hp.quniform('learning_rate', 0, 0.05, 0.0001),
            'n_estimators': hp.choice('n_estimators', range(100, 1000,100)),
            'eta': hp.quniform('eta', 0.025, 0.5, 0.005),
            'max_depth':  hp.choice('max_depth', np.arange(2, 12, dtype=int)),
            'min_child_weight': hp.quniform('min_child_weight', 1, 9, 0.025),
            'subsample': hp.quniform('subsample', 0.5, 1, 0.005),
            'gamma': hp.quniform('gamma', 0.5, 1, 0.005),
            'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.005),
            'eval_metric': 'auc',
            'objective': 'binary:logistic',
            'booster': 'gbtree',
            'tree_method': 'exact',
            'silent': 1,
            'missing': None
        }
 
best = fmin(fn=hyperopt_xgb_score, space=space_xgb, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)'''

"def hyperopt_xgb_score(params):\n    clf = XGBClassifier(**params)\n    current_score = cross_val_score(clf, x_train,y_train,cv=StratifiedKFold()).mean()\n    print(current_score, params)\n    return current_score \n \nspace_xgb = {\n            'learning_rate': hp.quniform('learning_rate', 0, 0.05, 0.0001),\n            'n_estimators': hp.choice('n_estimators', range(100, 1000,100)),\n            'eta': hp.quniform('eta', 0.025, 0.5, 0.005),\n            'max_depth':  hp.choice('max_depth', np.arange(2, 12, dtype=int)),\n            'min_child_weight': hp.quniform('min_child_weight', 1, 9, 0.025),\n            'subsample': hp.quniform('subsample', 0.5, 1, 0.005),\n            'gamma': hp.quniform('gamma', 0.5, 1, 0.005),\n            'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.005),\n            'eval_metric': 'auc',\n            'objective': 'binary:logistic',\n            'booster': 'gbtree',\n            'tree_method': 'exact',\n            'silent': 1,\n        

In [472]:
'''params = space_eval(space_xgb, best)
params'''

'params = space_eval(space_xgb, best)\nparams'

In [473]:
#catBoost Tuning
'''def hyperopt_cat_score(params):
  clf = CatBoostClassifier(**params)
  current_score = cross_val_score(clf,x_train,y_train, cv=StratifiedKFold(n_splits=10)).mean()
  print(current_score, params)
  return current_score 
 
space_cat = {
            'learning_rate': hp.quniform('learning_rate', 0.03, 0.001,0.01,0.1,0.2,0.3),
            'iterations': hp.quniform('iterations',250,100,500,1000,2500),
            'depth':  hp.quniform('depth',3,1,2,6,4,5,7,8,9,10),
            'l2_leaf_reg': hp.quniform('l2_leaf_reg',3,1,5,10,100),
            'border_count': hp.quniform('border_count',32,5,10,20,50,100,200),
            'ctr_border_count': hp.quniform('ctr_border_count',50,5,10,20,100,200),
            'thread_count':hp.quniform('thread_count',4)
            }

 
 
best = fmin(fn=hyperopt_cat_score, space=space_cat, algo=tpe.suggest, max_evals=10)
print('best:')
print(best)'''

"def hyperopt_cat_score(params):\n  clf = CatBoostClassifier(**params)\n  current_score = cross_val_score(clf,x_train,y_train, cv=StratifiedKFold(n_splits=10)).mean()\n  print(current_score, params)\n  return current_score \n \nspace_cat = {\n            'learning_rate': hp.quniform('learning_rate', 0.03, 0.001,0.01,0.1,0.2,0.3),\n            'iterations': hp.quniform('iterations',250,100,500,1000,2500),\n            'depth':  hp.quniform('depth',3,1,2,6,4,5,7,8,9,10),\n            'l2_leaf_reg': hp.quniform('l2_leaf_reg',3,1,5,10,100),\n            'border_count': hp.quniform('border_count',32,5,10,20,50,100,200),\n            'ctr_border_count': hp.quniform('ctr_border_count',50,5,10,20,100,200),\n            'thread_count':hp.quniform('thread_count',4)\n            }\n\n \n \nbest = fmin(fn=hyperopt_cat_score, space=space_cat, algo=tpe.suggest, max_evals=10)\nprint('best:')\nprint(best)"

In [474]:
'''params = space_eval(space_cat, best)
params'''

'params = space_eval(space_cat, best)\nparams'

In [475]:
'''#cat_dims=[46]
model = CatBoostClassifier(loss_function='Logloss')
grid = {'depth':[3,1,2,6,4,5,7,8,9,10],
          'iterations':[250,100,500,1000,2500],
          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], 
          'l2_leaf_reg':[3,1,5,10,100],
          
          }

grid_search_result = model.grid_search(grid, 
                                       X=x_train, 
                                       y=y_train,
                                       cv=StratifiedKFold(n_splits=10), 
                                       plot=True)'''

"#cat_dims=[46]\nmodel = CatBoostClassifier(loss_function='Logloss')\ngrid = {'depth':[3,1,2,6,4,5,7,8,9,10],\n          'iterations':[250,100,500,1000,2500],\n          'learning_rate':[0.03,0.001,0.01,0.1,0.2,0.3], \n          'l2_leaf_reg':[3,1,5,10,100],\n          \n          }\n\ngrid_search_result = model.grid_search(grid, \n                                       X=x_train, \n                                       y=y_train,\n                                       cv=StratifiedKFold(n_splits=10), \n                                       plot=True)"

In [476]:
#models
#cat_dims=[46]
etc=ExtraTreesClassifier(n_estimators=400,
                                     max_features=49,
                                     criterion= 'entropy',
                                     min_samples_split=2,
                                     max_depth=49,
                                     min_samples_leaf=2, ## the default is 1 (note we use more than one sample for the split)
                                     class_weight='balanced_subsample',
                                     random_state=1,
                                     verbose=1)

        
rfc=RandomForestClassifier(random_state=0, n_jobs=-1, n_estimators=400, max_depth=4)
xgbc=XGBClassifier(booster= 'gbtree',colsample_bytree= 0.98,
                    eta= 0.37,eval_metric='auc',gamma= 0.89,
                    learning_rate= 0.001,max_depth= 5,min_child_weight=5.8,
                    missing= None,n_estimators=900,objective='binary:logistic',
                    silent= 1,subsample= 0.55,tree_method='exact')
          
lgbc=LGBMClassifier(boosting_type='gbdt', colsample_bytree=0.87,
               importance_type='split', learning_rate=0.001, max_depth=5,
               min_child_samples=20, min_child_weight=5.675, min_split_gain=0.0,
               n_estimators=900, n_jobs=-1, num_leaves=1486, objective='binary',
               random_state=None,silent=True,
               )
catc=CatBoostClassifier(iterations=2500,
                            loss_function ='Logloss',
                            depth=5,
                            l2_leaf_reg =1e-20,
                            eval_metric = 'AUC',
                            leaf_estimation_iterations = 10,
                            )

In [477]:
etc.fit(x_train, y_train)
rfc.fit(x_train, y_train)
xgbc.fit(x_train, y_train)
lgbc.fit(x_train, y_train)
catc.fit(x_train,y_train,cat_features=cat_dims)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:  2.3min finished


0:	total: 35.8ms	remaining: 1m 29s
1:	total: 65.4ms	remaining: 1m 21s
2:	total: 93.7ms	remaining: 1m 18s
3:	total: 122ms	remaining: 1m 16s
4:	total: 152ms	remaining: 1m 16s
5:	total: 180ms	remaining: 1m 14s
6:	total: 208ms	remaining: 1m 13s
7:	total: 235ms	remaining: 1m 13s
8:	total: 266ms	remaining: 1m 13s
9:	total: 294ms	remaining: 1m 13s
10:	total: 322ms	remaining: 1m 12s
11:	total: 350ms	remaining: 1m 12s
12:	total: 379ms	remaining: 1m 12s
13:	total: 408ms	remaining: 1m 12s
14:	total: 441ms	remaining: 1m 13s
15:	total: 471ms	remaining: 1m 13s
16:	total: 500ms	remaining: 1m 12s
17:	total: 529ms	remaining: 1m 12s
18:	total: 567ms	remaining: 1m 13s
19:	total: 594ms	remaining: 1m 13s
20:	total: 623ms	remaining: 1m 13s
21:	total: 655ms	remaining: 1m 13s
22:	total: 684ms	remaining: 1m 13s
23:	total: 713ms	remaining: 1m 13s
24:	total: 740ms	remaining: 1m 13s
25:	total: 767ms	remaining: 1m 12s
26:	total: 796ms	remaining: 1m 12s
27:	total: 823ms	remaining: 1m 12s
28:	total: 853ms	remaining:

In [478]:
'''from sklearn.model_selection import KFold, cross_val_score
n_folds = 5 # number of folds
def get_cv_scores(model, X, y, print_scores=True):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X) # create folds
    rocauc = (cross_val_score(model, X, y, scoring="roc_auc", cv = kf)) # get rmse
    if print_scores:
        print('ROC AUC:',rocauc.mean() ,rocauc.std())
    return [rocauc]'''

'from sklearn.model_selection import KFold, cross_val_score\nn_folds = 5 # number of folds\ndef get_cv_scores(model, X, y, print_scores=True):\n    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X) # create folds\n    rocauc = (cross_val_score(model, X, y, scoring="roc_auc", cv = kf)) # get rmse\n    if print_scores:\n        print(\'ROC AUC:\',rocauc.mean() ,rocauc.std())\n    return [rocauc]'

In [479]:
'''for model in [etc, rfc, xgbc, lgbc,catc]:
    get_cv_scores(model,x_train, y_train)'''

'for model in [etc, rfc, xgbc, lgbc,catc]:\n    get_cv_scores(model,x_train, y_train)'

In [480]:
y_pred1 =etc.predict_proba(x_test)[:,1]
y_pred2 =rfc.predict_proba(x_test)[:,1]
y_pred3 =xgbc.predict_proba(x_test)[:,1]
y_pred4 =lgbc.predict_proba(x_test)[:,1]
y_pred5=catc.predict_proba(x_test)[:,1]


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed:    3.0s finished


In [481]:
#averaged_models = AveragedModels([rdgc,knn,adac, gbrc,etc, rfc, xgbc, lgbc])
#averaged_models=(y_pred1+y_pred2+y_pred3+y_pred4+y_pred5+y_pred6+y_pred7+y_pred8)/8
averaged_models=(y_pred1+y_pred2+y_pred3+y_pred4+y_pred5)/5

In [482]:
#weithed_averaged=(0.200*0.198*y_pred1+0.200*y_pred2+0.200*y_pred3+0.200*y_pred4+0.202*y_pred5)/5

In [483]:
#Prediction for averaging model our Submission file
my_submission = pd.DataFrame({'Application ID':Application_id, 'default_status':averaged_models})
my_submission.to_csv('submission_ayanlola_average_reformwithcat_model1.csv', index=False)

In [484]:
'''from sklearn.base import BaseEstimator
from sklearn.base import RegressorMixin,ClassifierMixin
from sklearn.base import TransformerMixin
from sklearn.base import clone
class WeightedAveragedModels(BaseEstimator,ClassifierMixin, TransformerMixin):
    def __init__(self, models, weights):
        self.models = models
        self.weights = weights
        assert sum(self.weights)==1
        
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)
        return self
    
    def predict(self, X):
        predictions = np.column_stack([
            model.predict.proba(X)[:,1] for model in self.models_
        ])
        return np.sum(predictions*self.weights, axis=1)'''

'from sklearn.base import BaseEstimator\nfrom sklearn.base import RegressorMixin,ClassifierMixin\nfrom sklearn.base import TransformerMixin\nfrom sklearn.base import clone\nclass WeightedAveragedModels(BaseEstimator,ClassifierMixin, TransformerMixin):\n    def __init__(self, models, weights):\n        self.models = models\n        self.weights = weights\n        assert sum(self.weights)==1\n        \n    def fit(self, X, y):\n        self.models_ = [clone(x) for x in self.models]\n        \n        # Train cloned base models\n        for model in self.models_:\n            model.fit(X, y)\n        return self\n    \n    def predict(self, X):\n        predictions = np.column_stack([\n            model.predict.proba(X)[:,1] for model in self.models_\n        ])\n        return np.sum(predictions*self.weights, axis=1)'

In [485]:
'''perf_rmse_scores=[0.829,0.816,0.826,0.825,0.835]
total=sum(perf_rmse_scores)
percent_list=[]

for i in range(len(perf_rmse_scores)):
  per= 100* perf_rmse_scores[i]/total
  percent_list.append(per)
percent_list'''

'perf_rmse_scores=[0.829,0.816,0.826,0.825,0.835]\ntotal=sum(perf_rmse_scores)\npercent_list=[]\n\nfor i in range(len(perf_rmse_scores)):\n  per= 100* perf_rmse_scores[i]/total\n  percent_list.append(per)\npercent_list'

In [486]:
'''weighted_average_model.fit(x_train,y_train)
weighted_y_pred = averaged_model1.predict_proba(x_valid)[:,1]'''

'weighted_average_model.fit(x_train,y_train)\nweighted_y_pred = averaged_model1.predict_proba(x_valid)[:,1]'

In [487]:
weighted_average_model = WeightedAveragedModels([etc,rfc,xgbc,lgbc,catc],[0.201,0.198,0.200,0.200,0.202])
get_cv_scores(weighted_average_model, x_train, y_train)

AssertionError: ignored

In [ ]:
'''#Prediction for averaging model our Submission file
my_submission = pd.DataFrame({'Application ID':Application_id, 'default_status':weighted_average_model})
my_submission.to_csv('submission_ayanlola_weightaveraged_reformwithcat_model.csv', index=False)'''

In [ ]:
#get_cv_scores(weighted_average_model, x_train, y_train)

In [ ]:
'''
perf_rmse_scores=[751.682,752.220,741.768,]
total=sum(perf_rmse_scores)
percent_list=[]

for i in range(len(perf_rmse_scores)):
  per= 100* perf_rmse_scores[i]/total
  percent_list.append(per)'''

In [ ]:
#predictions2'''=0.334*weighted_pred +.335 *xgb_pred +.330 * lgb_pred

In [ ]:
#Prediction for averaging model our Submission file
#my_submission = pd.DataFrame({'Application ID':Application_id, 'default_status':predictions2})
#my_submission.to_csv('submission_ayanlola_averaged_reformwithcat_model.csv', index=False)